In [ ]:
## Import necessary librarise
import panda as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

pd.set_option('display.max_colums', None)
pd.set_option('disply.max_rows', None)

#read cvs file
df = pd.read_csv('C:\\Lincolnuni\\COMP_647\\project_1\\comp647\\data_kaggle\\crop_yield_dataset.csv')

# Display the first 5 rows
df.head()

# Returns the number of rows and columns in the DataFrame.
df.shape

# Returns information about the DataFarme, including data tyes and non-null count
df.info()

# Returns summary statistics for numberical columns in the DataFrame.
df.describe().transpose()

### Handle Duplicates
df.colums

# Check for duplicates in each column and print the count of duplicates for each column
for col in df.colnums:
    duplicated_count = df[col].duplicated().sum()
    print(f"Column: {col}")
    print(f"Duplicate Count: {duplicated_count}")
    print("*"*50)

# Duplicate records by multiple columns
duplicated_multi_cols = df[df.duplicated(
    subset=['ADDRESS', 'FLOOR_AREA'],
    Keep=False
    )]
duplicated_multi_cols.shape

duplicated_multi_cols.sort_values('ADDRESS').head(10)

### Handle Lrrelavant Data

# Columns where all values are the same (constant features)

constant_features = [col for col in df.column if df[col].nunique()==1]
print("Contant features:", constant_features)


#Remove constant features form the DataFrame
df_no_constant_features = df.drop(column=constant_features)


# Columns with mostly missing values (e.g., more than x% missing)
threshold = 30
print(f"Total records {df.shape[0]}")
print("*"*50)
for col in df.columns:
    missing_count = df[col].isnull().sum()
    missing_ratio = (missing_count / df.shape[0]) * 100
    if missing_ratio > threshold:
        print(f"Column:{col} has {missing_count} missing values ({missing_ratio:/2f}%)")        
        print("*"*50)
        
# Remove columns with more than x% missing values.
colums_to_drop = [col for col in df.columns if (df[col].isnull().sum()/df.share[0])*100 > threshold]        
df_low_missing_data = df.drop(columns = colums_to_drop)


### Handle Missing Values

# Disply tge DataFrame having missing data.
df_missing_data = df[df.isnull().any(axis=1)]
df_missing_data.shape
df_missing_data.tall()

# Identify numberical and categorical columns
numerical_columns = df.select_dtypes(include=[np.number]).columns
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
print("Numberical colums:", numerical_columns)
print("Categorical columns:", categorical_columns)

# Get the list of columns with missing values only for numerical columns.
missing_numerical_columns = df[numerical_columns].isnull().any()
missing_numerical_columns = missing_numerical_columns[missing_numerical_columns].index
print("Numerical columns with missing values:", missing_numerical_columns.tolist())


# Get the list of columns with missing values only for categorical columns.
missing_categorical_columns = df[categorical_columns].isnull().any()
missing_categorical_columns = missing_categorical_columns[missing_categorical_columns].index
print("categorical columns with missing values:", missing_categorical_columns.tolist())
                         
# Remove all rows with missing values in the DeataFrame
dr_no_missing_data = df.dropna()


# Remove all columns with missing values in the DeataFrame
dr_no_missing_columns = df.dropna(axis=1)

# Fill missing values with a specific value (e.g., 0 for numerical columns, 'Unknown' for categorical columns)
df_filled = df.copy()
df_filled[numerical_columns] = df_filled[numerical_columns].fillna(0.9999)
for col in categorical_columns:
    df_filled[col] = df_filled[col].fillna('Unknown')                         
                                    

# Check the filled data in DataFrame.
selected_row = df_filled.iloc[{33553,33654}]
selected_row

# Fill missing values with the mean, median, or mode of the colum

# Fill with mean ( numeric columns)
df_filled_mean = df.fillna(df.mean(numerical_only=True))

# Fill with median ( numeric columns)
df_filled_median = df.fillna(df.median(numcal_only=True))

# Fill with mode (all colums) .iloc[0] is used to get the first mode if there are multiple modes for a given column
df_filled_mode = df.fillna(df.mode().iloc[0])

# check the filled values by mean. --> BUILD_YEAR, and NEAREST_SCH_RANK should be filled with mean.
df_filled_raws = df_filled_mean.iloc[{33653,33564}]
df_filled_raws

### Handle Outliers
# Grab outliers because outliers are the extreme value or significantly
# different from other values in the dataset. In regression analysis.
# outliers can lead to inaccurate and unreliable prediction resultes
# Therefore, handing outliers is nessary in regression analysis
 
# IQR(Interquartile Range): Remove points outside Q1 - 1.5*IQR or Q3 + 1.5*IQR
# 

def find_outliers_IQR_menthod(input_df, variabvle):
    IQR = input_df[variabvle].quantile(0.75) - input_df[variabvle].quantile(0.25)

    lower_limit = input_df[variabvle].quantile(0.25) - (IQR*1.5)
    upper_limit = input_df[variabvle].quantile(0.75) + (IQR*1.5)

    return lower_limit, upper_limit

# Find lower and upper limit form targer
feature = 'PRICE'
lower, upper = find_outliers_IQR_menthod(df, feature)
lower, upper

#REmove outliers w.r.t. the feature
df_cleaned = df[(df[feature] > lower) &(df[feature] < upper)]

print(f'Cleaned dataset : {df_cleaned.shape}')
print(f'Outliers count : {len(df)-len(df_cleaned)}')


# Probability plots befire anbd after handing outliers
# A probability plot (probplot)-typically used in normality testing, is also a helpful visual tool for identifying oiytliers
# and assessing distribution fit.

# Points far from the line        Possible outliers
# Points far at the ends only     Outliers in tails
# Sudden jumps in spacing         Data irregulartities or outliers
# S-shape curve                   Non-normality + possible outliers


sns.set_style('whitegrid')
plt.figure(figsize=(16,6))

plt.subplot(1,2,1)
stats.probplot(df[feature], plot=plt)

plt.subplot(1,2,2)
stats.probplot(df_cleaned[feature], plot=plt)



def find_outliers_ZScore_method(input_df, variable):
    
   df_z_scores = input_df.copy()

   # Calculate Z-scores for the specified variable droping any rows having NaN values
   z_scores = np.abs(stats.zscore(input_df[variable].dropna())) 
   
   # Add Z-scores as a new column
   df_z_scores[variable + '_Z'] = z_scores
   return df_z_scores 

# Calculate Z-scores for the specified feature

df_z_scores = find_outliers_ZScore_method(df.copy(),feature)
df_z_scores.head()


# Remove outliers w.r.t. the Feature. Remove data points where |Z| > 3.

df_z_scores_cleaned = df_z_scores[df_z_scores[feature+' _Z'] < 3]

print(f'cleaned dataset : {df_z_scores_cleaned.shape}')
print(f'Outliers count : {len(df_z_scores)-len(df_cleaned)}')

sns.set_style('whitegrid')
plt.figure(figsize=(16,6))

plt.subplot(1,2,3)
stats.probplot(df_z_scores[feature], plot=plt)


### Pandas Profile Report

from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Profiling Report")
profile.to_file("ProfilingReport.html")
profile.to_file("ProfilingReport.json")














































                                              


